# Set up Essentials

In [0]:
from google.colab import drive , files
import os, os.path
import pandas as pd
from pandas.io.json import json_normalize 
import json
import requests
import numpy as np

## **Mount Drive and Operate from within said folder**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
%cd /content/drive/'My Drive'/Colab_Notebooks/mzalendo

/content/drive/My Drive/Colab_Notebooks/mzalendo


In [0]:
persons = "http://info.mzalendo.com/media_root/popolo_json/persons.json"
organization = "http://info.mzalendo.com/media_root/popolo_json/organizations.json"
memberships = "http://info.mzalendo.com/media_root/popolo_json/memberships.json"
pombola = "http://info.mzalendo.com/media_root/popolo_json/memberships.json"
candidates = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTR3hgRnOrPi6Opkr3Z622_onNYEtzds51TxqS1-VBEcDqP8O0nLjr1XwRkQYCGLrusNL6nJLyxtQbe/pub?gid=671749755&single=true&output=csv"
elected = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTR3hgRnOrPi6Opkr3Z622_onNYEtzds51TxqS1-VBEcDqP8O0nLjr1XwRkQYCGLrusNL6nJLyxtQbe/pub?gid=655626479&single=true&output=csv"


## Work from file with nested jsons'

In [0]:
with open(pombola) as data_file:    
    data = json.load(data_file) 

# data = json_normalize("pombola.json")

In [0]:
df = json_normalize(data)

In [0]:
df.head()

,areas,events,organizations,persons,posts
0,"[{'area_type': 'CON', 'id': 'mapit:4812', 'ide...","[{'classification': 'legislative period', 'end...","[{'category': 'other', 'classification': 'Unkn...","[{'contact_details': [], 'id': 'core_person:78...",[]


In [0]:
df.columns

Index(['areas', 'events', 'organizations', 'persons', 'posts'], dtype='object')

In [0]:
len(df['persons'][0])

13267

In [0]:
df_P = pd.DataFrame(df['persons'][0])

In [0]:
df_P.head(3)

,contact_details,id,identifiers,images,memberships,name,other_names,sort_name,birth_date,gender,death_date,summary,email,biography,national_identity,family_name,given_name,honorific_suffix,honorific_prefix
0,[],core_person:7848,"[{'identifier': 'abbey-gedow-abdikadir', 'sche...",[{'url': 'http://info.mzalendo.com/media_root/...,"[{'area': None, 'end_date': '2013-03-04', 'id'...",Abbey Gedow Abdikadir,[],Abdikadir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],core_person:8251,"[{'identifier': 'al-amin-ahmed', 'scheme': 'po...",[{'url': 'http://info.mzalendo.com/media_root/...,"[{'id': 'core_position:21297', 'identifiers': ...",Al Amin Ahmed,[],Ahmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],core_person:4435,"[{'identifier': 'abdi-ahmed-ali', 'scheme': 'p...",[{'url': 'http://info.mzalendo.com/media_root/...,"[{'id': 'core_position:13754', 'identifiers': ...",Abdi Ahmed Ali,[],Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Collect All Candidates data

In [0]:
all = pd.read_csv(candidates)
elected = pd.read_csv(elected)

In [0]:
all.head(3)

,name,party,party-id,p-abbrv,gender,position,county,county-id,constituency,constituency-id,ward,ward-id
0,JOHN EKURU LONGOGGY AUKOT,THIRDWAY ALLIANCE KENYA,76,TAK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN
1,MOHAMED ABDUBA DIDA,ALLIANCE FOR REALCHANGE,55,ARK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN
2,SHAKHALAGA KHWA JIRONGO,UNITED DEMOCRATIC PARTY,64,UDP,Male,president,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
all.shape

(14758, 12)

## Isolate Elected legislators

In [0]:
all['elected'] = np.where(all['name'].isin(elected['name'].str.upper()),"elected-2017", "not-elected")


In [0]:
all.head(3)

,name,party,party-id,p-abbrv,gender,position,county,county-id,constituency,constituency-id,ward,ward-id,elected
0,JOHN EKURU LONGOGGY AUKOT,THIRDWAY ALLIANCE KENYA,76,TAK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN,not-elected
1,MOHAMED ABDUBA DIDA,ALLIANCE FOR REALCHANGE,55,ARK,Male,president,NaN,NaN,NaN,NaN,NaN,NaN,not-elected
2,SHAKHALAGA KHWA JIRONGO,UNITED DEMOCRATIC PARTY,64,UDP,Male,president,NaN,NaN,NaN,NaN,NaN,NaN,not-elected


In [0]:
# all.to_csv("./ke_legislators.csv")

# Persons Section

Access the json file

In [0]:
data_p = pd.read_json(persons)

In [0]:
data_p.head(3)

,_id,contact_details,id,identifiers,images,name,other_names,sort_name,birth_date,gender,death_date,summary,email,biography,national_identity,family_name,given_name,honorific_suffix,honorific_prefix
0,core_person:7848,[],core_person:7848,"[{'identifier': 'abbey-gedow-abdikadir', 'sche...",[{'url': 'http://info.mzalendo.com/media_root/...,Abbey Gedow Abdikadir,[],Abdikadir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,core_person:8251,[],core_person:8251,"[{'identifier': 'al-amin-ahmed', 'scheme': 'po...",[{'url': 'http://info.mzalendo.com/media_root/...,Al Amin Ahmed,[],Ahmed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,core_person:4435,[],core_person:4435,"[{'identifier': 'abdi-ahmed-ali', 'scheme': 'p...",[{'url': 'http://info.mzalendo.com/media_root/...,Abdi Ahmed Ali,[],Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
p_columns = ['id', 'name','contact_details', 'images', 'birth_date', 'gender', 'death_date',
             'summary', 'email', 'biography', 'national_identity']

data_p = data_p[p_columns]

In [0]:
data_p.head(3)

,id,name,contact_details,images,birth_date,gender,death_date,summary,email,biography,national_identity,phone
0,7848,Abbey Gedow Abdikadir,[],http://info.mzalendo.com/media_root/images/2BB...,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,8251,Al Amin Ahmed,[],http://info.mzalendo.com/media_root/images/C17...,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,4435,Abdi Ahmed Ali,[],http://info.mzalendo.com/media_root/images/863...,NaN,NaN,NaN,NaN,None,NaN,NaN,None


Clean up *id* and *images*  columns

In [0]:
def clean_id(x):
  if pd.notnull(x):
    id = x.split(":")[1]
    return id

def get_img_url(x):
  if pd.notnull(x):
    url = x[0].get('url')
    return url


get emails from contacts columns

In [0]:
def get_email(x):
  for i in x:
    contacts_type = i.get('type')
    if contacts_type == "email":
      email = i.get('value')
      return email

def get_phone(x):
  for i in x:
    contacts_type = i.get('type')
    if contacts_type == "phone":
      phone = i.get('value')
      return phone

def get_address(x):
  for i in x:
    contacts_type = i.get('type')
    if contacts_type == "address":
      address = i.get('value')
      return address






In [0]:
# data_p.to_csv('ke_persons.csv')

In [0]:
# Access the ID's only
data_p['id'] = data_p['id'].apply(lambda x: clean_id(x))
# get url from dictionary
data_p['images'] = data_p['images'].apply(lambda x: get_img_url(x))
# add email
data_p['email'] = data_p['contact_details'].apply(lambda x: get_email(x)  )
# add phone no.'s
data_p['phone'] = data_p['contact_details'].apply(lambda x: get_phone(x)  )

# Organizations Section

In [0]:
data_org = pd.read_json(organization)
data_org.head(3)

,_id,category,classification,contact_details,id,identifiers,name,slug,founding_date,dissolution_date
0,core_organisation:2471,other,Unknown,[],core_organisation:2471,[],Coffee Board of Kenya,coffee-board-kenya,NaN,NaN
1,core_organisation:2472,other,Unknown,[],core_organisation:2472,[],Aberdare Spa and Safari Lodge,aberdare-spa-and-safari-lodge,NaN,NaN
2,core_organisation:2475,other,Unknown,[],core_organisation:2475,[],Co-Operative Bank,co-operative-bank,NaN,NaN


In [0]:
org_columns = ['id','name', 'category', 'classification','contact_details',
               'founding_date', 'dissolution_date']
data_org = data_org[org_columns]
data_org.head(3)

,id,name,category,classification,contact_details,founding_date,dissolution_date
0,core_organisation:2471,Coffee Board of Kenya,other,Unknown,[],NaN,NaN
1,core_organisation:2472,Aberdare Spa and Safari Lodge,other,Unknown,[],NaN,NaN
2,core_organisation:2475,Co-Operative Bank,other,Unknown,[],NaN,NaN


Clean columns

In [0]:

# Access the ID's only
data_org['id'] = data_org['id'].apply(lambda x: clean_id(x))
# add email
data_org['email'] = data_org['contact_details'].apply(lambda x: get_email(x)  )
# add phone no.'s
data_org['phone'] = data_org['contact_details'].apply(lambda x: get_phone(x)  )
# add addreses
data_org['address'] = data_org['contact_details'].apply(lambda x: get_address(x)  )


data_org.head(3)

,id,name,category,classification,contact_details,founding_date,dissolution_date,email,phone,address
0,2471,Coffee Board of Kenya,other,Unknown,[],NaN,NaN,None,None,None
1,2472,Aberdare Spa and Safari Lodge,other,Unknown,[],NaN,NaN,None,None,None
2,2475,Co-Operative Bank,other,Unknown,[],NaN,NaN,None,None,None


In [0]:
# data_org.to_csv('KE_mzalendo_organizations.csv')

# Memberships

In [8]:
data_mb = pd.read_json(memberships)

data_mb.head(3)

,_id,area,end_date,id,identifiers,organization_id,person_id,role,start_date,legislative_period_id
0,core_position:20499,None,2013-03-04,core_position:20499,[{'identifier': '2BBD157E-3569-E211-9497-00505...,core_organisation:513,core_person:7848,Aspirant Ward Representative,2013-02-08,NaN
1,core_position:20498,NaN,NaN,core_position:20498,[],core_organisation:15,core_person:7848,Member,2013-02-08,NaN
2,core_position:29626,NaN,NaN,core_position:29626,[],core_organisation:1783,core_person:7848,Coalition Member,2013-02-13,NaN


In [10]:
len(data_mb)

44330

In [0]:
# clean Pos ID's 
data_mb['id'] = data_mb['id'].apply(lambda x: clean_id(x))
# org ID
data_mb['organization_id'] = data_mb['organization_id'].apply(lambda x: clean_id(x))
# person's ID
data_mb['person_id'] = data_mb['person_id'].apply(lambda x: clean_id(x))

In [0]:
mb_columns = ['id','organization_id','person_id','role','area', 
                   'end_date','start_date','identifiers','legislative_period_id']

data_mb = data_mb[mb_columns]

In [13]:
data_mb.head(3)

,id,organization_id,person_id,role,area,end_date,start_date,identifiers,legislative_period_id
0,20499,513,7848,Aspirant Ward Representative,None,2013-03-04,2013-02-08,[{'identifier': '2BBD157E-3569-E211-9497-00505...,NaN
1,20498,15,7848,Member,NaN,NaN,2013-02-08,[],NaN
2,29626,1783,7848,Coalition Member,NaN,NaN,2013-02-13,[],NaN


Define Memberships regions by 'Name' and 'Region-Type'

In [0]:
def get_region(x):
  if pd.notnull(x):
    region = x['name']
    return region

def get_area_type(x):
  if pd.notnull(x):
    area_type = x['area_type']
    return area_type

def get_term(x):
  if pd.notnull(x):
    term = x['session']['slug']
    return term


In [0]:
# data_mb.to_csv('ke_mzalendo_memberships.csv')

In [14]:
data_mb['area'][44320]

{'area_type': 'DIS',
 'id': 'mapit:5038',
 'identifier': 'http://info.mzalendo.com/mapit/area/5038',
 'name': 'Nyandarua',
 'session': {'end_date': '2017-07-16',
  'id': 3,
  'mapit_generation': 3,
  'name': 'Senate 2013-2017',
  'slug': 's2013',
  'start_date': '2013-03-05'}}